# My Playground for Neural Network

In [842]:
from utils import *
from stratified import *
from run import *
from neuralnetwork import *

In [843]:
def importhousedata():
    house = importfile('hw3_house_votes_84.csv', ',')
    housecategory = {}
    for i in house[0]:
        housecategory[i] = 'categorical'
    housecategory["class"] = 'class'
    housedata = np.array(house[1:]).astype(float)
    return housedata, housecategory
def importwinedata():
    wine = importfile('hw3_wine.csv', '\t')
    winecategory = {}
    for i in wine[0]:
        winecategory[i] = 'numerical'
    winecategory["# class"] = 'class'
    winedata = np.array(wine[1:]).astype(float)
    return winedata, winecategory
def importcmcdata():
    cmc = importfile('cmc.data', ',')
    cmccategory = {"Wife's age":"numerical","Wife's education":"categorical",
    "Husband's education":"categorical","Number of children ever born":"numerical",
    "Wife's religion":"binary","Wife's now working?":"binary",
    "Husband's occupation":"categorical","Standard-of-living index":"categorical",
    "Media exposure":"binary","Contraceptive method used":"class"}
    cmcdata = np.array(cmc).astype(int)
    return cmcdata, cmccategory
def importcancerdata():
    cancer = importfile('hw3_cancer.csv', '\t')
    cancercategory = {}
    for i in cancer[0]:
        cancercategory[i] = 'numerical'
    cancercategory["Class"] = 'class'
    cancerdata = np.array(cancer[1:]).astype(float)
    return cancerdata, cancercategory

housedata, housecategory = importhousedata()
winedata, winecategory = importwinedata()
cmcdata,cmccategory = importcmcdata()
cancerdata, cancercategory = importcancerdata()
example_category = {'ece': 'numerical','cs': 'categorical','phy': 'categorical','math': 'categorical','musi': 'categorical','grade': 'class'}
#example_data = np.array([[100,0,1,1,0,0],[123,1,0,1,1,1],[44,1,0,0,0,2],[92,1,0,1,0,3],[73,0,1,0,0,2]])
example_data = np.array([[10,0,1,1,0,0],[23,1,0,1,1,1],[44,1,0,0,0,2],[92,1,0,1,0,3],[73,0,1,0,0,2],[122,0,1,1,0,0],[124,1,0,1,1,1],[44,1,0,0,0,2],[40,1,0,1,0,3],[73,0,1,0,0,2]])

In [844]:
ohe_house_data, ohe_house_category = onehotencoder(housedata, housecategory)
ohe_wine_data, ohe_wine_category = onehotencoder(winedata,winecategory)
ohe_cmc_data, ohe_cmc_category = onehotencoder(cmcdata,cmccategory)
ohe_example_data, ohe_example_category = onehotencoder(example_data,example_category)
ohe_cancer_data, ohe_cancer_category = onehotencoder(cancerdata,cancercategory)

In [845]:
n_ohe_house_data = normalizetrain(ohe_house_data,ohe_house_category)[0]
n_ohe_wine_data = normalizetrain(ohe_wine_data,ohe_wine_category)[0]
n_ohe_cmc_data = normalizetrain(ohe_cmc_data,ohe_cmc_category)[0]
n_ohe_example_data = normalizetrain(ohe_example_data,ohe_example_category)[0]
n_ohe_cancer_data = normalizetrain(ohe_cancer_data,ohe_cancer_category)[0]

In [846]:
def g(x): # sigmoid function
    return np.exp(x) / (1 + np.exp(x))

In [847]:
def transposelistoflist(l):
    newlistoflist = []
    for i in range(len(l[0])):
        newlist = []
        for j in range(len(l)):
            newlist.append(l[j][i])
        newlistoflist.append(newlist)
    return newlistoflist

In [848]:
def gradientD(weights_list,deltalist,attributelist,biasterm=True):
    gradlist = []
    for i in range(len(weights_list)):
        attributenow = attributelist[i]
        deltanow = np.array([deltalist[i]]).T
        dotproduct = deltanow*attributenow
        # print('dotshape',dotproduct.shape)
        gradlist.append(dotproduct)
    return gradlist

In [849]:
def g(x): # sigmoid function
    return 1/(1 + np.exp(-x))

In [850]:
# Forward propagation vectorized
def neural_network(normed_hotted_data,ohe_category,weights_list,biasterm=True, minibatchk = 435, lambda_reg = 0.08, learning_rate = 0.0001):
    normed_ohe_copy = normed_hotted_data.copy()
    np.random.shuffle(normed_ohe_copy)
    splitted = np.array_split(normed_ohe_copy, minibatchk)
    
    inputcategory, outputcategory = [],[]
    inputindex, outputindex = [],[]
    n = 0
    for i in ohe_category:
        if ohe_category[i] != 'class_numerical':
            inputcategory.append(i) # name of the input category
            inputindex.append(n) # index of the input category
        else:
            outputcategory.append(i) # name of the output category
            outputindex.append(n) # index of the output category
        n += 1
    
    b = 1 if biasterm else 0
    
    for onebatch in splitted:
        onebatch = onebatch.T
        input_data = onebatch[inputindex].T
        output_data = onebatch[outputindex].T
        # input_data_mean = onebatch[inputindex].mean(axis=1)
        output_data = onebatch[outputindex].T

        # forward propagation
        instance_index = 0
        j = 0
        listofgradient = []
        for one_instance in input_data:
            current_layer_a = np.append(1,one_instance) if b == 1 else one_instance
            # input layer is the current layer
            current_layer_index = 0
            output_real = output_data[instance_index]
            attributesnobias = [one_instance]
            attributeswbias = [current_layer_a]
            for theta in weights_list:
                z = np.dot(theta,current_layer_a)
                a = g(z)
                current_layer_a = np.append(1,a) if (b == 1) and (current_layer_index+1 != len(weights_list)) else a
                attributesnobias.append(a)
                attributeswbias.append(current_layer_a)
                current_layer_index += 1

            output_predict = current_layer_a # the last attribute is the output for this batch.
            instance_index += 1
            j += costfunction(output_real,output_predict)

            # calculate delta blame (back propagation)
            listofdelta = blame(output_real,output_predict,weights_list, attributeswbias)
            thisgradient = gradientD(weights_list,listofdelta,attributeswbias,biasterm)
            listofgradient.append(thisgradient)
        
        gradientP = [lambda_reg*t for t in weights_list]
        grad_D_transpose = transposelistoflist(listofgradient)
        grad_D_sum = [np.sum(t,axis=0) for t in grad_D_transpose]
        gradients_batch = []
        for i in range(len(grad_D_sum)):
            gradients_batch.append((grad_D_sum[i] + gradientP[i])*(1/instance_index))
        
        j /= (instance_index+1)
        s = sumofweights(weights_list,bias=b)*lambda_reg/(2*(instance_index+1))
        allj = j+s # total cose with regularization

        # update weights
        for i in range(len(weights_list)):
            weights_list[i] -= learning_rate*gradients_batch[i]

    return weights_list, allj, j

In [851]:
layerdataparameter = [5,5,5,5,5,5] # [First layer, Second layer, Third layer], not including the bias and input & output layer
initweights = initialize_weights(ohe_house_category,layerdataparameter)
kk,jj,pj = neural_network(n_ohe_house_data,ohe_house_category,initweights)

In [852]:
j_previous = 99
epsilon = 80
weight, j_now, purej = neural_network(n_ohe_house_data,ohe_house_category,initweights)
epsilon = j_previous - j_now
j_previous = j_now
smallestj = j_now
count = 0
while epsilon > 0.00001 and j_now >= smallestj:
    print(count)
    count+=1
    weight, j_now, purej = neural_network(n_ohe_house_data,ohe_house_category,weight)
    print('purej',purej)
    print('j now is',j_now)
    smallestj = min(smallestj,j_now)
    epsilon = abs(j_previous - j_now)
    print(epsilon)
    j_previous = j_now

0
purej 0.6373585774994234
j now is 2.6464630062182892
0.027337541403822385
1
purej 0.6271640809823287
j now is 2.621387848259815
0.025075157958474037
2
purej 0.6175979907442336
j now is 2.5988147304473705
0.022573117812444643
3
purej 0.6081165271358391
j now is 2.578119229548215
0.020695500899155395
4
purej 1.265714992133981
j now is 3.2263430127646586
0.6482237832164435
5
purej 0.5923997296559385
j now is 2.545372135537003
0.6809708772276557
6
purej 0.5844119118726503
j now is 2.5313262248305572
0.014045910706445675
7
purej 0.5775259603768985
j now is 2.520008713775241
0.01131751105531631
8
purej 0.5711259272267133
j now is 2.5105880140874817
0.00942069968775927
9
purej 0.5648409594285982
j now is 2.50252439901546
0.00806361507202169
10
purej 0.559394141940274
j now is 2.496484355363295
0.006040043652165039
11
purej 0.5539497092878849
j now is 2.4913713793281596
0.005112976035135386
12
purej 1.178652702790391
j now is 3.1172953707813544
0.6259239914531949
13
purej 1.1694565901889733


In [ ]:
print(j_now)
correctweight = weight

0.5199060581554483


In [ ]:
# def predictusingnetwork(inputdata, category, weight):


In [ ]:
layernow = 3
layernext = 2
init_weight = np.random.rand(layernext,layernow) * 2 - 1
a_now = np.random.rand(layernow,1)
print(init_weight)
print(a_now)
c = np.dot(init_weight,a_now)
print(c)

[[ 0.80209967 -0.66025338 -0.863001  ]
 [ 0.66490296  0.20841189  0.37449872]]
[[0.15182842]
 [0.90023871]
 [0.87945888]]
[[-1.23157802]
 [ 0.61792784]]
